# 1-D Maxwell's Equations (Unit 2.4)

[Peter Monk: "Finite Elements for Maxwell's Equations"]

## Magnetostatic field generated by a permanent magnet

magnetic flux $B$, magnetic field $H$, given magnetization $M$:

$$B=\mu(H+M), \hspace{1em} \text{div }B=0,\hspace{1em}\text{curl }H=0$$

Introducing a vector-potential $A$ such that $B=\text{curl}A$, and putting equations together we get (how?)
$$\text{curl }\mu^{−1} \text{curl }A=\text{curl }M$$

In weak form: Find $A\in H(curl)$ such that

$$\int \mu^{−1} \text{curl }A \text{ curl }v=\int M \text{ curl }v \ \forall v \in H(curl)$$

Usually, the permeability $\mu$
is given as $\mu=\mu_r \mu_0$, with $\mu_0=4\pi 10^{−7}$ the permeability of vacuum.

## Notes:
I'm not really sure how to proceed with this one in 1-D.  Skip for now...

In [1]:
from ngsolve import *
from netgen.csg import *
import netgen.gui
%gui tk

Geometric model and meshing of a bar magnet:

In [4]:
box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"))
geo.Add (magnet.mat("magnet").maxh(1))
mesh = Mesh(geo.GenerateMesh(maxh=2))


In [5]:
mesh.GetMaterials(), mesh.GetBoundaries()

(('air', 'magnet'),
 ('outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'default',
  'default',
  'default'))

Define space, forms and preconditioner. To obtain a regular system matrix, we add a very small $L_2$ term

In [6]:
fes = HCurl(mesh, order=3, dirichlet="outer", nograds=True)
print ("ndof =", fes.ndof)
u = fes.TrialFunction()
v = fes.TestFunction()

from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1000 if mat== "magnet" else 1 
                            for mat in mesh.GetMaterials()]) 

a = BilinearForm(fes)
a += SymbolicBFI(1/(mu0*mur) * curl(u) * curl(v) + 1e-6/(mu0*mur)*u*v)
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((1,0,0))
f += SymbolicLFI(mag*curl(v), definedon=mesh.Materials("magnet"))

ndof = 88638


Assemble system and setup preconditioner using (shared memory) task-parallelization:

In [10]:
with TaskManager():
    a.Assemble()
    f.Assemble()

In [11]:
gfu = GridFunction(fes)

inv = CGSolver(a.mat, c.mat)
gfu.vec.data = inv * f.vec
Draw (curl(gfu), mesh, "B-field")

Finally, declare GridFunction and solve by preconditioned CG iteration:

we forgot to curve the mesh, this can be done by

In [9]:
mesh.Curve(3)

Go back and assemble and solve again ...

To visualize, you may want to zoom inside the orthobrick to see the cylinder